Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Spleen 3D segmentation with MONAI

This tutorial demonstrates how MONAI can be used in conjunction with the [PyTorch Lightning](https://github.com/PyTorchLightning/pytorch-lightning) framework.

We demonstrate use of the following MONAI features:
1. Transforms for dictionary format data.
1. Loading Nifti images with metadata.
1. Add channel dim to the data if no channel dimension.
1. Scaling medical image intensity with expected range.
1. Croping out a batch of balanced images based on  the positive / negative label ratio.
1. Cache IO and transforms to accelerate training and validation.
1. Use of a a 3D UNet model, Dice loss function, and mean Dice metric for a 3D segmentation task.
1. The sliding window inference method.
1. Deterministic training for reproducibility.

The Spleen dataset can be downloaded from http://medicaldecathlon.com/.

![spleen](http://medicaldecathlon.com/img/spleen0.png)

Target: Spleen  
Modality: CT  
Size: 61 3D volumes (41 Training + 20 Testing)  
Source: Memorial Sloan Kettering Cancer Center  
Challenge: Large ranging foreground size

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/3d_segmentation/spleen_segmentation_3d_lightning.ipynb)

## Setup environment

In [5]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q pytorch-lightning~=2.0
%matplotlib inline

/bin/bash: line 1: python: command not found
^C
ERROR: Operation cancelled by user
/bin/bash: line 1: python: command not found


## Setup imports

In [1]:
import pytorch_lightning
from pytorch_lightning.callbacks import ModelCheckpoint
from monai.utils import set_determinism
from monai.transforms import (
    AsDiscrete,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureType,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, DataLoader
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import nibabel as nib

print_config()

/home/gasyna/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-06-03 19:46:56.748680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


MONAI version: 1.3.0
Numpy version: 1.23.5
Pytorch version: 2.0.0+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.8.0
Pillow version: 9.0.1
Tensorboard version: 2.12.2
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.1+cu117
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.7
pandas version: 1.5.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED 

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpl1i1sq2c


In [3]:
data_dir = '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath'

## Download dataset

Downloads and extracts the dataset.
The dataset comes from http://medicaldecathlon.com/.

In [4]:
resource = "https://zenodo.org/records/10069289/files/AeroPath.zip?download=1"
md5 = "3fd5106c175c85d60eaece220f5dfd87"

compressed_file = os.path.join(root_dir, "AeroPath.zip")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## Define the LightningModule

The LightningModule contains a refactoring of your training code. The following module is a refactoring of the code in `spleen_segmentation_3d.ipynb`:

In [12]:
class Net(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH,
        )
        self.loss_function = DiceLoss(to_onehot_y=True, softmax=True)
        self.post_pred = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(argmax=True, to_onehot=2)])
        self.post_label = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(to_onehot=2)])
        self.dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
        self.best_val_dice = 0
        self.best_val_epoch = 0
        self.validation_step_outputs = []

    def forward(self, x):
        return self._model(x)

    def prepare_data(self):
        # # set up the correct data path
        # pattern = os.path.join(data_dir, '**/*_CT_HR_label_airways.nii.gz')
        # train_labels = sorted(glob.glob(pattern, recursive=True))

        # pattern = os.path.join(data_dir, '**/*_CT_HR.nii.gz')
        # train_images = sorted(glob.glob(pattern, recursive=True))

        pattern = os.path.join('overlapping_labels', '**/quadrant_1_*.nii.gz')
        train_labels = sorted(glob.glob(pattern, recursive=True))

        pattern = os.path.join('overlapping_quadrants', '**/quadrant_1_*_CT_HR.nii.gz')
        train_images = sorted(glob.glob(pattern, recursive=True))

        data_dicts = [
            {"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)
        ]
        train_files, val_files = data_dicts[:-9], data_dicts[-9:]

        # set deterministic training for reproducibility
        set_determinism(seed=0)

        # define the data transforms
        train_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-57,
                    a_max=164,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
                # randomly crop out patch samples from
                # big image based on pos / neg ratio
                # the image centers of negative samples
                # must be in valid image area

                RandCropByPosNegLabeld(
                    keys=["image", "label"],
                    label_key="label",
                    spatial_size=(64, 64, 64),
                    pos=1,
                    neg=1,
                    num_samples=4,
                    image_key="image",
                    image_threshold=0,
                ),

                # user can also add other random transforms
                #                 RandAffined(
                #                     keys=['image', 'label'],
                #                     mode=('bilinear', 'nearest'),
                #                     prob=1.0,
                #                     spatial_size=(96, 96, 96),
                #                     rotate_range=(0, 0, np.pi/15),
                #                     scale_range=(0.1, 0.1, 0.1)),
            ]
        )
        val_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-57,
                    a_max=164,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
            ]
        )
                    

        # we use cached datasets - these are 10x faster than regular datasets
        self.train_ds = CacheDataset(
            data=train_files,
            transform=train_transforms,
            cache_rate=1.0,
            num_workers=4,
        )
        self.val_ds = CacheDataset(
            data=val_files,
            transform=val_transforms,
            cache_rate=1.0,
            num_workers=4,
        )

        self.infer_ds = CacheDataset(
            data=val_files,
            cache_rate=1.0,
            num_workers=4,
        )

    #         self.train_ds = monai.data.Dataset(
    #             data=train_files, transform=train_transforms)
    #         self.val_ds = monai.data.Dataset(
    #             data=val_files, transform=val_transforms)

    def train_dataloader(self):
        train_loader = DataLoader(
            self.train_ds,
            batch_size=2,
            shuffle=True,
            num_workers=4,
            collate_fn=list_data_collate,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.val_ds, batch_size=1, num_workers=4)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self._model.parameters(), 1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        output = self.forward(images)
        loss = self.loss_function(output, labels)
        tensorboard_logs = {"train_loss": loss.item()}
        return {"loss": loss, "log": tensorboard_logs}

    
    def validation_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(images, roi_size, sw_batch_size, self)
        loss = self.loss_function(outputs, labels)
        outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        labels = [self.post_label(i) for i in decollate_batch(labels)]
        self.dice_metric(y_pred=outputs, y=labels)
        d = {"val_loss": loss, "val_number": len(outputs)}
        self.validation_step_outputs.append(d)
        return d
    
    def evaluate_single_model(self, model, file_paths):
        # Load and preprocess NIfTI files
        input_data = [self.load_and_preprocess_nifti(file_path) for file_path in file_paths]

        # Perform inference using the model
        model_output = [self.perform_inference(model, data) for data in input_data]

        return model_output
    
    def load_and_preprocess_nifti(self, file_path):
        img = nib.load(file_path)
        data = img.get_fdata()
        data = torch.DoubleTensor(data)  # Convert data to type Double
        # Apply any preprocessing steps here, such as normalization
        # Remember to ensure the data shape matches the input shape expected by your models
        return data


    
    def perform_inference(self, model, data):
        # Perform inference using the model
        with torch.no_grad():
            data = torch.DoubleTensor(data)  # Convert data to type Double
            model_output = model(data.unsqueeze(0))
        return model_output

    def on_validation_epoch_end(self):
        val_loss, num_items = 0, 0
        for output in self.validation_step_outputs:
            val_loss += output["val_loss"].sum().item()
            num_items += output["val_number"]
        mean_val_dice = self.dice_metric.aggregate().item()
        self.dice_metric.reset()
        mean_val_loss = torch.tensor(val_loss / num_items)
        tensorboard_logs = {
            "val_dice": mean_val_dice,
            "val_loss": mean_val_loss,
        }
        if mean_val_dice > self.best_val_dice:
            self.best_val_dice = mean_val_dice
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current mean dice: {mean_val_dice:.4f}"
            f"\nbest mean dice: {self.best_val_dice:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.validation_step_outputs.clear()  # free memory
        self.log('val_dice', mean_val_dice, on_step=False, on_epoch=True, prog_bar=True, logger=True) # log

        return {"log": tensorboard_logs}

In [ ]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_dice',
    dirpath=os.path.join(root_dir, 'checkpoints'),  # Directory to save checkpoints
    filename='best-checkpoint_whole_64',  # Filename prefix for saving checkpoints
    save_top_k=1,  # Save only the best checkpoint
    mode='max',  # `min` for minimizing the metric, `max` for maximizing
    verbose=True,  # Log a message when saving the best checkpoint
)

## Create overlapping quadrants

In [34]:
from natsort import natsorted

def create_overlapping_quadrants(image_shape, overlap_percentage=0.0):
    x, y, z = image_shape
    overlap_z = int(z * overlap_percentage)
    if z < 3 * overlap_z:
        return [(0, z)]
    mid_z = z // 2
    return [
        (0, mid_z + overlap_z),
        (mid_z - overlap_z, z)
    ]

def load_nifti_file(file_path):
    return nib.load(file_path)

def save_nifti_file(data, affine, file_path):
    img = nib.Nifti1Image(data, affine)
    nib.save(img, file_path)

def process_file(file_path, output_dir):
    img = load_nifti_file(file_path)
    data = img.get_fdata()
    print("input shape: ", data.shape)
    affine = img.affine
    quadrants = create_overlapping_quadrants(data.shape)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, (start, end) in enumerate(quadrants):
        quadrant_data = data[:, :, start:end]
        quadrant_file_path = os.path.join(output_dir, f"quadrant_{i+1}_{os.path.basename(file_path)}")
        save_nifti_file(quadrant_data, affine, quadrant_file_path)
        print(f"Saved: {quadrant_file_path}")
        print(f'Quadrant {i} shape: {quadrant_data.shape}')


# Example Usage
pattern = os.path.join(data_dir, '**/*_CT_HR_label_airways.nii.gz')
train_labels = natsorted(glob.glob(pattern, recursive=True))

pattern = os.path.join(data_dir, '**/*_CT_HR.nii.gz')
train_images = natsorted(glob.glob(pattern, recursive=True))

# for idx, (image_name, label_name) in enumerate(zip(train_images, train_labels)):
#     output_dir = f'nonoverlapping_quadrants/{idx}'
#     process_file(image_name, output_dir)
#     output_dir = f'nonoverlapping_labels/{idx}'
#     process_file(label_name, output_dir)



for idx, (image_name, label_name) in enumerate(zip(train_images, train_labels)):
    output_dir = f'nonoverlapping_quadrants/{idx + 1}'
    process_file(image_name, output_dir)
    output_dir = f'nonoverlapping_labels/{idx + 1}'
    process_file(label_name, output_dir)
    
    print('')
    print('###########')
    print('')


input shape:  (512, 512, 767)
Saved: nonoverlapping_quadrants/1/quadrant_1_1_CT_HR.nii.gz
Quadrant 0 shape: (512, 512, 383)
Saved: nonoverlapping_quadrants/1/quadrant_2_1_CT_HR.nii.gz
Quadrant 1 shape: (512, 512, 384)
input shape:  (512, 512, 767)
Saved: nonoverlapping_labels/1/quadrant_1_1_CT_HR_label_airways.nii.gz
Quadrant 0 shape: (512, 512, 383)
Saved: nonoverlapping_labels/1/quadrant_2_1_CT_HR_label_airways.nii.gz
Quadrant 1 shape: (512, 512, 384)

###########

input shape:  (512, 512, 829)
Saved: nonoverlapping_quadrants/2/quadrant_1_2_CT_HR.nii.gz
Quadrant 0 shape: (512, 512, 414)
Saved: nonoverlapping_quadrants/2/quadrant_2_2_CT_HR.nii.gz
Quadrant 1 shape: (512, 512, 415)
input shape:  (512, 512, 829)
Saved: nonoverlapping_labels/2/quadrant_1_2_CT_HR_label_airways.nii.gz
Quadrant 0 shape: (512, 512, 414)
Saved: nonoverlapping_labels/2/quadrant_2_2_CT_HR_label_airways.nii.gz
Quadrant 1 shape: (512, 512, 415)

###########

input shape:  (512, 512, 714)
Saved: nonoverlapping_quad

In [33]:
from natsort import natsorted
import glob

# Assuming 'pattern' is defined
pattern = "/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/*/*_CT_HR.nii.gz"

# Get the list of files
train_images = glob.glob(pattern, recursive=True)

# Sort the files naturally
sorted_train_images = natsorted(train_images)

# Print the sorted list
for idx, image in enumerate(sorted_train_images):
    print(idx)
    print(image)

0
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/1/1_CT_HR.nii.gz
1
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/2/2_CT_HR.nii.gz
2
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/3/3_CT_HR.nii.gz
3
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/4/4_CT_HR.nii.gz
4
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/5/5_CT_HR.nii.gz
5
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/6/6_CT_HR.nii.gz
6
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/7/7_CT_HR.nii.gz
7
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/8/8_CT_HR.nii.gz
8
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/9/9_CT_HR.nii.gz
9
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/10/10_CT_HR.nii.gz
10
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/11/11_CT_HR.nii.gz
11
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/12/12_CT_HR.nii.gz
12
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/13/13_CT_HR.nii.gz
13
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/14/14_CT_HR.nii.gz
14
/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/15/15_CT_HR.nii.gz
15
/home

In [31]:
pattern = os.path.join(data_dir, '**/*_CT_HR.nii.gz')
train_images = sorted(glob.glob(pattern, recursive=True))
train_images

['/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/1/1_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/10/10_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/11/11_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/12/12_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/13/13_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/14/14_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/15/15_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/16/16_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/17/17_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/18/18_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/19/19_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/2/2_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/20/20_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation/AeroPath/21/21_CT_HR.nii.gz',
 '/home/gasyna/RiSA_S3/3D_segmentation

In [ ]:

pattern = os.path.join('overlapping_labels', '**/quadrant_1_*.nii.gz')
train_labels = sorted(glob.glob(pattern, recursive=True))

pattern = os.path.join('overlapping_quadrants', '**/quadrant_1_*_CT_HR.nii.gz')
train_images = sorted(glob.glob(pattern, recursive=True))

In [7]:
img = 'nonoverlapping_quadrants/24/quadrant_2_7_CT_HR.nii.gz'
label = 'nonoverlapping_labels/24/quadrant_2_7_CT_HR_label_airways.nii.gz'
img = nib.load(img).get_fdata()
label = nib.load(label).get_fdata()

print(f'img shape: {img.shape}, label shape: {label.shape}')


# whole
# img shape: (512, 512, 723), label shape: (512, 512, 723)

# 1q
# img shape: (512, 512, 361), label shape: (512, 512, 361)

# 2q
# img shape: (512, 512, 362), label shape: (512, 512, 362)

img shape: (512, 512, 362), label shape: (512, 512, 362)


In [ ]:

for img, label in zip (train_images, train_labels):
    img = nib.load(img).get_fdata()
    label = nib.load(label).get_fdata()

    print(f'img shape: {img.shape}, label shape: {label.shape}')

In [ ]:
import matplotlib.pyplot as plt

# quadrant1 = nib.load('quadrant1.nii.gz').get_fdata()
# quadrant2 = nib.load('quadrant2.nii.gz').get_fdata()
# quadrant3 = nib.load('quadrant3.nii.gz').get_fdata()
# quadrant4 = nib.load('quadrant4.nii.gz').get_fdata()


def plot_slice(data, title):
    plt.figure()
    plt.imshow(data[data.shape[0] // 2, :, :], cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

# plot_slice(quadrant1, "Quadrant 1")
# plot_slice(quadrant2, "Quadrant 2")
# plot_slice(quadrant3, "Quadrant 3")
# plot_slice(quadrant4, "Quadrant 4")

## Run the training

In [ ]:
# initialise the LightningModule
net = Net()

# set up loggers and checkpoints
log_dir = os.path.join(root_dir, "logs")
tb_logger = pytorch_lightning.loggers.TensorBoardLogger(save_dir=log_dir)

# initialise Lightning's trainer.
trainer = pytorch_lightning.Trainer(
    devices=[0],
    max_epochs=600,
    logger=tb_logger,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=1,
    log_every_n_steps=16,
)

# train
trainer.fit(net)

In [ ]:
print(f"train completed, best_metric: {net.best_val_dice:.4f} " f"at epoch {net.best_val_epoch}")

In [ ]:
import torch
from monai.networks.nets import UNet


# Load the model weights from the checkpoint file
checkpoint_path = 'best-checkpoint.ckpt'
model = Net.load_from_checkpoint(checkpoint_path)

# Set the model to evaluation mode
model.eval()


## Model Ensembling

In [ ]:
# Instantiate the Net class

model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_48_0.6382 at epoch 338.ckpt')
model.eval()


file_path = '/home/gasyna/RiSA_S3/3D_segmentation/overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz'
img = nib.load(file_path)
data = img.get_fdata()

data = torch.DoubleTensor(data)  # Convert data to type Double


with torch.no_grad():
    y_hat = model(data)

RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [512, 512, 536]

### Model inference (works)

In [7]:
# Model inference

import torch
import nibabel as nib
from monai.transforms import (
    Spacingd, ScaleIntensityRanged, CropForegroundd, Orientationd, EnsureTyped, Compose
)
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch

# Load the trained model
model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_96_0.6293 at epoch 340.ckpt')
model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define the transformations for new data
infer_transforms = Compose(
    [
        Orientationd(keys="image", axcodes="RAS"),
        # Spacingd(keys="image", pixdim=(1.5, 1.5, 2.0), mode=("bilinear")),
        ScaleIntensityRanged(keys="image", a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
        # CropForegroundd(keys="image", source_key="image"),
        EnsureTyped(keys="image"),
    ]
)

# Load and preprocess the new data
file_path = 'overlapping_quadrants/18/quadrant_1_26_CT_HR.nii.gz'
img = nib.load(file_path)
data = img.get_fdata()

# Manually add the channel dimension
data = data[None, ...]
data_raw = data
# Create a dictionary with the image
data_dict = {"image": data}

# Apply transformations
data_dict = infer_transforms(data_dict)

# Convert data to tensor and add batch dimension
data = torch.tensor(data_dict["image"]).unsqueeze(0).to(model.device)


# Function to run inference
def infer_on_single_image(model, data):
    model.eval()
    with torch.no_grad():
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(data, roi_size, sw_batch_size, model)
        post_processed_outputs = [model.post_pred(i) for i in decollate_batch(outputs)]
    return post_processed_outputs

# Run inference
predictions = infer_on_single_image(model, data)

# Process and save the predictions as needed
for i, prediction in enumerate(predictions):
    # Save or process each prediction here
    # For example, save as NIfTI file
    prediction_np = prediction.cpu().numpy()
    pred_img = nib.Nifti1Image(prediction_np, img.affine)
    # nib.save(pred_img, f"prediction_{i}.nii.gz")
    nib.save(pred_img, "prediction.nii.gz")

print("input raw shape: ", data_raw.shape)
print("input shape: ", data.shape)
print("output shape: ", pred_img.shape)
print("Inference complete.")


RuntimeError: applying transform <monai.transforms.io.dictionary.LoadImaged object at 0x7c4230bad900>

In [7]:
# Model inference

import torch
import nibabel as nib
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd, ScaleIntensityRanged, CropForegroundd, EnsureTyped, Compose
)
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch

# Load the trained model
model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_96_0.6293 at epoch 340.ckpt')
model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define the transformations for validation and inference
infer_transforms = Compose(
    [
        LoadImaged(keys="image"),
        EnsureChannelFirstd(keys="image"),
        Orientationd(keys="image", axcodes="RAS"),
        Spacingd(keys="image", pixdim=(1.5, 1.5, 2.0), mode=("bilinear")),
        ScaleIntensityRanged(keys="image", a_min=-57, a_max=164, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys="image", source_key="image"),
        EnsureTyped(keys="image"),
    ]
)

# Load and preprocess the new data
file_path = 'overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz'
data_dict = {"image": file_path}

# Apply transformations
data_dict = infer_transforms(data_dict)
data = data_dict["image"]

data_raw = data
# Convert data to tensor and add batch dimension
data = torch.tensor(data).unsqueeze(0).to(model.device)

# Function to run inference
def infer_on_single_image(model, data):
    model.eval()
    with torch.no_grad():
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(data, roi_size, sw_batch_size, model)
        post_processed_outputs = [model.post_pred(i) for i in decollate_batch(outputs)]
    return post_processed_outputs

# Run inference
predictions = infer_on_single_image(model, data)

# Process and save the predictions as needed
for i, prediction in enumerate(predictions):
    # Save or process each prediction here
    # For example, save as NIfTI file
    prediction_np = prediction.cpu().numpy()
    pred_img = nib.Nifti1Image(prediction_np, img.affine)
    # nib.save(pred_img, f"prediction_{i}.nii.gz")
    nib.save(pred_img, "prediction.nii.gz")

print("input raw shape: ", data_raw.shape)
print("input shape: ", data.shape)
print("output shape: ", pred_img.shape)
print("Inference complete.")


/home/gasyna/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/tmp/ipykernel_81960/684908623.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).unsqueeze(0).to(model.device)


NameError: name 'img' is not defined

## Dice score

In [13]:
label = nib.load('overlapping_labels/0/quadrant_1_1_CT_HR_label_airways.nii.gz')
# image = nib.load('overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz')
image = nib.load('prediction.nii.gz')

label = label.get_fdata()
image = image.get_fdata()

image = image[0, :, :]

print(f'img shape: {image.shape}, label shape: {label.shape}')

img shape: (234, 205, 135), label shape: (512, 512, 536)


In [89]:
# Model inference and Dice score calculation

import torch
import nibabel as nib
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd, ScaleIntensityRanged, CropForegroundd, EnsureTyped, Compose, AsDiscrete
)
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.metrics import DiceMetric

# Load the trained model
# model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_64_0.6679 at epoch: 498.ckpt')
model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_2Q_spatial_size_64_0.8143 at epoch 499.ckpt')
# moel = Net.load_from_checkpoint('checkpoints/best-checkpoint_whole_64_0.8023 at epoch: 427.ckpt')
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the transformations for validation and inference
common_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)

# Load and preprocess the new data and the label

# # 1Q
# file_path = 'nonoverlapping_quadrants/7/quadrant_1_7_CT_HR.nii.gz'
# label_path = 'nonoverlapping_labels/7/quadrant_1_7_CT_HR_label_airways.nii.gz'  # Update with your actual label file path

# 2Q
# file_path = 'nonoverlapping_quadrants/7/quadrant_2_7_CT_HR.nii.gz'
# label_path = 'nonoverlapping_labels/7/quadrant_2_7_CT_HR_label_airways.nii.gz'  # Update with your actual label file path

# # WHOLE
file_path = 'AeroPath/7/7_CT_HR_label_lungs.nii.gz'
label_path = 'AeroPath/7/7_CT_HR_label_lungs.nii.gz'

data_dict = {"image": file_path, "label": label_path}


# Apply transformations
data_dict = common_transforms(data_dict)
data = data_dict["image"]
label = data_dict["label"]

# Convert data and label to tensors and add batch dimension


data = torch.tensor(data).unsqueeze(0).to(device)
label = torch.tensor(label).unsqueeze(0).to(device)

data_nii = nib.load(file_path).get_fdata()
label_nii = nib.load(label_path).get_fdata()

print("input nii gz shape: ", data_nii.shape)
print("input shape: ", data.shape)

# Function to run inference
def infer_on_single_image(model, data):
    model.eval()
    with torch.no_grad():
        roi_size = (64, 64, 64)
        sw_batch_size = 4
        outputs = sliding_window_inference(data, roi_size, sw_batch_size, model)
        post_processed_outputs = [model.post_pred(i) for i in decollate_batch(outputs)]
    return post_processed_outputs

# Run inference
predictions = infer_on_single_image(model, data)

# Process and save the predictions as needed
for i, prediction in enumerate(predictions):
    # Save or process each prediction here
    # For example, save as NIfTI file
    prediction_np = prediction.cpu().numpy()
    pred_img = nib.Nifti1Image(prediction_np, nib.load(file_path).affine)
    nib.save(pred_img, "whole_prediction.nii.gz")

# Convert predictions and labels to binary format if necessary
post_pred = Compose([AsDiscrete(argmax=True, to_onehot=2)])
post_label = Compose([AsDiscrete(to_onehot=2)])

# Apply the transformations directly to the tensors and ensure they are on the same device
prediction_tensor = post_pred(prediction.to(device))
label_tensor = post_label(label.to(device))

# Compute Dice score
dice_metric = DiceMetric(include_background=True, reduction="mean")
dice_metric(y_pred=prediction_tensor, y=label_tensor)
dice_score = dice_metric.aggregate().item()
dice_metric.reset()

print("input shape: ", data.shape)
print("output shape: ", pred_img.shape)
print(f"Dice Score: {dice_score:.4f}")
print("Inference complete.")

pred_nii = nib.load('prediction.nii.gz').get_fdata()

print("output nii gz shape: ", pred_nii.shape)

#NONOVERLAPPING
# 1Q
# input nii gz shape:  (512, 512, 361)
# input shape:  torch.Size([1, 1, 259, 192, 91])
# input shape:  torch.Size([1, 1, 259, 192, 91])
# output shape:  (2, 259, 192, 91)
# Dice Score: 0.5217

# 2Q
# input nii gz shape:  (512, 512, 362)
# input shape:  torch.Size([1, 1, 259, 188, 91])
# input shape:  torch.Size([1, 1, 259, 188, 91])
# output shape:  (2, 259, 188, 91)
# Dice Score: 0.6679

# WHOLE
# input nii gz shape:  (512, 512, 723)
# input shape:  torch.Size([1, 1, 259, 259, 182])
# input shape:  torch.Size([1, 1, 259, 259, 182])
# output shape:  (2, 259, 259, 182)
# Dice Score: 0.9771

# OVERLAPPING
# 1Q
# input shape:  torch.Size([1, 1, 259, 194, 127])
# output shape:  (2, 259, 194, 127)
# Dice Score: 0.5829

# 2Q
# input shape:  torch.Size([1, 1, 259, 190, 127])
# output shape:  (2, 259, 190, 127)
# Dice Score: 0.6390

#NONOVERLAPPING WITHOUT SPACINGD
# 1Q

/tmp/ipykernel_121231/3556753813.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).unsqueeze(0).to(device)
/tmp/ipykernel_121231/3556753813.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).unsqueeze(0).to(device)


input nii gz shape:  (512, 512, 241)
input shape:  torch.Size([1, 1, 240, 240, 151])
input shape:  torch.Size([1, 1, 240, 240, 151])
output shape:  (2, 240, 240, 151)
Dice Score: 0.9864
Inference complete.
output nii gz shape:  (2, 259, 259, 182)


## Interpolate ensemble

In [62]:
import numpy as np

Q1 = nib.load('1Qprediction.nii.gz').get_fdata()
Q2 = nib.load('2Qprediction.nii.gz').get_fdata()

print(f'Q1 shape: {Q1.shape}')
print(f'Q2 shape: {Q2.shape}')

Q1_new = np.concatenate((Q1, Q2), axis=3)

print(f'Q1_new shape: {Q1_new.shape}')

Q1 shape: (2, 259, 192, 91)
Q2 shape: (2, 259, 188, 91)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 192 and the array at index 1 has size 188

In [90]:
import numpy as np
from scipy.ndimage import zoom

# Define a function to interpolate predictions to match the dimensions of the WHOLE model
def interpolate_predictions(predictions, target_shape):
    # Extract the original shape of the predictions
    original_shape = predictions.shape
    # Compute the scaling factors for interpolation along each axis
    scale_factors = [t / o for t, o in zip(target_shape, original_shape)]
    # Interpolate the predictions using zoom
    interpolated_predictions = zoom(predictions, zoom=scale_factors, mode='nearest')
    return interpolated_predictions

# Assuming you have predictions from both 1Q and 2Q models
predictions_1Q = nib.load('1Qprediction.nii.gz').get_fdata()  # Get predictions from 1Q model
predictions_2Q = nib.load('2Qprediction.nii.gz').get_fdata()  # Get predictions from 2Q model

whole = nib.load('whole_prediction.nii.gz').get_fdata()


# Assuming target shape is the shape of the WHOLE model
target_shape = (2, 259, 259, 91)  # Adjust as necessary
target_shape = (2, 240, 240, 75)  # Adjust as necessary

# Interpolate predictions from 1Q and 2Q models to match the target shape
interpolated_predictions_1Q = interpolate_predictions(predictions_1Q, target_shape)
target_shape = (2, 240, 240, 76)  # Adjust as necessary

interpolated_predictions_2Q = interpolate_predictions(predictions_2Q, target_shape)

# Combine predictions using averaging or another suitable method


merged =np.concatenate((interpolated_predictions_1Q, interpolated_predictions_2Q), axis=3)

print(f'merged: {merged.shape}')
print(f'whole: {whole.shape}')

ensemble_mean = (merged + whole) / 2  # Averaging for ensemble
ensemble_maximum = np.maximum(merged,whole)

# Now you have the ensemble predictions with the shape of the WHOLE model


merged: (2, 240, 240, 151)
whole: (2, 240, 240, 151)


In [98]:
# Apply the transformations directly to the tensors and ensure they are on the same device
prediction = torch.tensor(merged)

prediction_tensor = post_pred(prediction.to(device))
label_tensor = post_label(label.to(device))

# Compute Dice score
dice_metric = DiceMetric(include_background=True, reduction="mean")
dice_metric(y_pred=prediction_tensor, y=label_tensor)
dice_score = dice_metric.aggregate().item()
dice_metric.reset()

print(dice_score)

0.9864209294319153
0.9864209294319153


In [ ]:
# Apply the transformations directly to the tensors and ensure they are on the same device
merged = torch.tensor((np.maximum(merged,whole)))

prediction_tensor = post_pred(merged.to(device))
label_tensor = post_label(label.to(device))

# Compute Dice score
dice_metric = DiceMetric(include_background=True, reduction="mean")
dice_metric(y_pred=prediction_tensor, y=label_tensor)
dice_score = dice_metric.aggregate().item()
dice_metric.reset()

print(dice_score)

In [37]:
# Model inference and Dice score calculation

import torch
import nibabel as nib
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd, ScaleIntensityRanged, CropForegroundd, EnsureTyped, Compose, AsDiscrete
)
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.metrics import DiceMetric
import numpy as np

# Load the trained models (add more as needed for ensembling)
models = [
    Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_96_0.6293 at epoch 340.ckpt'),
    # Net.load_from_checkpoint('checkpoints/another_checkpoint.ckpt')
]

for model in models:
    model.eval()
    model.to('cuda' if torch.cuda.is_available() else 'cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the transformations for validation and inference
common_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)

# Load and preprocess the new data and the label
file_path = 'overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz'
label_path = 'overlapping_labels/0/quadrant_1_1_CT_HR_label_airways.nii.gz'  # Update with your actual label file path
data_dict = {"image": file_path, "label": label_path}

# Apply transformations
data_dict = common_transforms(data_dict)
data = data_dict["image"]
label = data_dict["label"]

# Save the original shape for padding
original_shape = data.shape[1:]  # exclude batch dimension

# Convert data and label to tensors and add batch dimension
data = torch.tensor(data).unsqueeze(0).unsqueeze(0).to(device)
label = torch.tensor(label).unsqueeze(0).to(device)
# Function to run inference
def infer_on_single_image(model, data):
    model.eval()
    with torch.no_grad():
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(data, roi_size, sw_batch_size, model)
        post_processed_outputs = [model.post_pred(i) for i in decollate_batch(outputs)]
    return post_processed_outputs

# Run inference on all models and aggregate the predictions
ensemble_predictions = []

for model in models:
    predictions = infer_on_single_image(model, data)
    ensemble_predictions.append(predictions[0].cpu().numpy())  # Assuming single prediction

# Average the predictions (or apply other ensembling techniques)
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Ensure the predictions are in the correct format
ensemble_predictions = torch.tensor(ensemble_predictions).unsqueeze(0).unsqueeze(0).to(device)

# Resize the ensemble predictions to the original shape if necessary
if ensemble_predictions.shape[2:] != original_shape:
    ensemble_predictions = torch.nn.functional.interpolate(
        ensemble_predictions, size=original_shape, mode='trilinear'
    ).squeeze(0).squeeze(0)

# Convert back to numpy for saving
ensemble_predictions = ensemble_predictions.cpu().numpy()

# Save the ensemble prediction
pred_img = nib.Nifti1Image(ensemble_predictions, nib.load(file_path).affine)
nib.save(pred_img, "ensemble_prediction.nii.gz")

# Convert predictions and labels to binary format if necessary
post_pred = Compose([AsDiscrete(argmax=True, to_onehot=2)])
post_label = Compose([AsDiscrete(to_onehot=2)])

# Apply the transformations directly to the tensors and ensure they are on the same device
prediction_tensor = post_pred(torch.tensor(ensemble_predictions).to(device))
label_tensor = post_label(label.to(device))

# Compute Dice score
dice_metric = DiceMetric(include_background=False, reduction="mean")
dice_metric(y_pred=prediction_tensor, y=label_tensor)
dice_score = dice_metric.aggregate().item()
dice_metric.reset()

print("input shape: ", data.shape)
print("output shape: ", pred_img.shape)
print(f"Dice Score: {dice_score:.4f}")
print("Inference complete.")


/tmp/ipykernel_364452/43230831.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data).unsqueeze(0).unsqueeze(0).to(device)
/tmp/ipykernel_364452/43230831.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).unsqueeze(0).to(device)


ValueError: Sequence must have length 4, got 3.

In [80]:
# Model inference and Dice score calculation

import torch
import nibabel as nib
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd, ScaleIntensityRanged, CropForegroundd, EnsureTyped, Compose, AsDiscrete, AddChannelD, RandSpatialCropD, SpatialPadD
)
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.metrics import DiceMetric
import numpy as np

# Load the trained models (add more as needed for ensembling)
models = [
    Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_96_0.6293 at epoch 340.ckpt'),
    # Net.load_from_checkpoint('checkpoints/another_checkpoint.ckpt')
]

for model in models:
    model.eval()
    model.to('cuda' if torch.cuda.is_available() else 'cpu')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the transformations for validation and inference
common_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-57,
            a_max=164,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
        AddChannelD(keys=["image", "label"]),
        RandSpatialCropD(keys=["image", "label"], roi_size=(160, 160, 160), random_center=True, random_size=False),
        SpatialPadD(keys=["image", "label"], spatial_size=(160, 160, 160))
    ]
)

# Load and preprocess the new data and the label
file_path = 'overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz'
label_path = 'overlapping_labels/0/quadrant_1_1_CT_HR_label_airways.nii.gz'  # Update with your actual label file path
data_dict = {"image": file_path, "label": label_path}

# Apply transformations
data_dict = common_transforms(data_dict)
data = data_dict["image"]
label = data_dict["label"]

# Save the original shape for padding
original_shape = data.shape[1:]  # exclude batch dimension

# Convert data and label to tensors and add batch dimension
data = torch.tensor(data).unsqueeze(0).to(device)
label = torch.tensor(label).unsqueeze(0).unsqueeze(0).to(device)

# Function to run inference
def infer_on_single_image(model, data):
    model.eval()
    with torch.no_grad():
        roi_size = (160, 160, 160)  # Use a 3D tuple for the spatial dimensions (depth, height, width)
        sw_batch_size = 4
        print(f"Input data shape: {data.shape}")
        outputs = sliding_window_inference(data, roi_size, sw_batch_size, model)
        print(f"Output data shape: {outputs.shape}")
        post_processed_outputs = [model.post_pred(i) for i in decollate_batch(outputs)]
    return post_processed_outputs

# Run inference on all models and aggregate the predictions
ensemble_predictions = []

for model in models:
    predictions = infer_on_single_image(model, data)
    ensemble_predictions.append(predictions[0].cpu().numpy())  # Assuming single prediction

# Average the predictions (or apply other ensembling techniques)
ensemble_predictions = np.mean(ensemble_predictions, axis=0)

# Ensure the predictions are in the correct format
ensemble_predictions = torch.tensor(ensemble_predictions).unsqueeze(0).unsqueeze(0).to(device)

# Resize the ensemble predictions to the original shape if necessary
if ensemble_predictions.shape[2:] != original_shape:
    ensemble_predictions = torch.nn.functional.interpolate(
        ensemble_predictions, size=original_shape, mode='trilinear', align_corners=False
    ).squeeze(0)

# Convert back to numpy for saving
ensemble_predictions = ensemble_predictions.cpu().numpy().squeeze(0)

# Save the ensemble prediction
pred_img = nib.Nifti1Image(ensemble_predictions, nib.load(file_path).affine)
nib.save(pred_img, "ensemble_prediction.nii.gz")

# Convert predictions and labels to binary format if necessary
post_pred = Compose([EnsureTyped(keys="pred"), AsDiscrete(argmax=True, to_onehot=2)])
post_label = Compose([EnsureTyped(keys="label"), AsDiscrete(to_onehot=2)])

# Apply the transformations directly to the tensors and ensure they are on the same device
prediction_tensor = post_pred({"pred": torch.tensor(ensemble_predictions).unsqueeze(0).unsqueeze(0).to(device)})["pred"]
label_tensor = post_label({"label": label.to(device)})["label"]

# Compute Dice score
dice_metric = DiceMetric(include_background=False, reduction="mean")
dice_metric(y_pred=prediction_tensor, y=label_tensor)
dice_score = dice_metric.aggregate().item()
dice_metric.reset()

print("input shape: ", data.shape)
print("output shape: ", pred_img.shape)
print(f"Dice Score: {dice_score:.4f}")
print("Inference complete.")


ImportError: cannot import name 'AddChannelD' from 'monai.transforms' (/home/gasyna/.local/lib/python3.10/site-packages/monai/transforms/__init__.py)

In [ ]:
# Instantiate the Net class

model = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_48_0.6382 at epoch 338.ckpt')
model.eval()


file_path = '/home/gasyna/RiSA_S3/3D_segmentation/overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz'
img = nib.load(file_path)
data = img.get_fdata()

data = torch.DoubleTensor(data)  # Convert data to type Double


with torch.no_grad():
    y_hat = model(data)

RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [512, 512, 536]

In [ ]:
# Instantiate the Net class
model = Net()

# Load the checkpoint for the single model
checkpoint_path = 'checkpoints/best-checkpoint_new_method_1Q_spatial_size_48_0.6382 at epoch 338.ckpt'
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])  # Assuming 'state_dict' is the key for model state_dict in the checkpoint

# Set the model to evaluation mode
model.eval()

# Define the paths to the NIfTI files
file_paths = [
    '/home/gasyna/RiSA_S3/3D_segmentation/overlapping_quadrants/0/quadrant_1_1_CT_HR.nii.gz',
    # Add paths for other files if applicable
]

# Perform inference using the single model
model_output = model.evaluate_single_model(model, file_paths)


In [ ]:
def load_and_preprocess_nifti(file_path):
    img = nib.load(file_path)
    data = img.get_fdata()
    # Apply any preprocessing steps here, such as normalization
    # Remember to ensure the data shape matches the input shape expected by your models
    return data


model = Net().double()  # Ensure the model is set to accept Double type input

# Set the model to evaluation mode
model.eval()



# Define the paths to the NIfTI files and load them
input_data = [load_and_preprocess_nifti(file_path) for file_path in file_paths]

# Convert input data to Double type
input_data = [torch.DoubleTensor(data) for data in input_data]



with torch.no_grad():
    # Ensure each data has a batch dimension
    input_data = [data.unsqueeze(0) for data in input_data]
    # Perform inference
    model_output = [model(data) for data in input_data]

In [ ]:
import torch

# Define the paths to the checkpoints for each model
checkpoint_paths = [
    'checkpoints/best-checkpoint_new_method_1Q_spatial_size_48_0.6382 at epoch 338.ckpt',
    'checkpoints/best-checkpoint_new_method_2Q_spatial_size_48_0.7714 at epoch 584.ckpt'
    # Add paths for other models if applicable
]

# Create instances of the Net class for each model
models = []
for checkpoint_path in checkpoint_paths:
    # Initialize the model
    model = Net()
    
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])  # Assuming 'state_dict' is the key for model state_dict in the checkpoint
    
    # Set the model to evaluation mode
    model.eval()
    
    model.prepare_data()
    model.evaluate()
    # Append the model to the list of models
    models.append(model)

    break
# Now the models list contains instances of the Net class loaded with the trained weights from checkpoints


In [ ]:
import nibabel as nib
import numpy as np
import torch


# Define a function to load and preprocess NIfTI files
def load_and_preprocess_nifti(file_path):
    img = nib.load(file_path)
    data = img.get_fdata()
    # Apply any preprocessing steps here, such as normalization
    # Remember to ensure the data shape matches the input shape expected by your models
    return torch.tensor(data).unsqueeze(0).float()  # Assuming input shape is (batch_size, channels, height, width, depth)

# Define a function to perform inference on NIfTI data using the ensemble of models
def perform_inference(models, input_data):
    model_outputs = []
    for model in models:
        model.eval()
        with torch.no_grad():
            output = model(input_data)
            model_outputs.append(output)
    # Aggregate the outputs from different models
    aggregated_output = torch.mean(torch.stack(model_outputs), dim=0)
    return aggregated_output

# Define the paths to the NIfTI files and load them
file_paths = ["path_to_file1.nii.gz", "path_to_file2.nii.gz", ...]
input_data = [load_and_preprocess_nifti(file_path) for file_path in file_paths]

# Perform inference using the ensemble of models
ensemble_output = [perform_inference(models, data) for data in input_data]

# Post-process the output predictions if necessary
# For example, you can convert the logits to probabilities using softmax

# Save or visualize the results
# You can save the output as NIfTI files or visualize them using plotting libraries like matplotlib or mayavi

## View training in tensorboard

Please uncomment the following cell to load tensorboard results.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$log_dir

## Check best model output with the input image and label

In [ ]:
net.eval()
device = torch.device("cuda:0")
net.to(device)
with torch.no_grad():
    for i, val_data in enumerate(net.val_dataloader()):
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        val_outputs = sliding_window_inference(val_data["image"].to(device), roi_size, sw_batch_size, net)
        # plot the slice [:, :, 80]
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"image {i}")
        plt.imshow(val_data["image"][0, 0, :, :, 80], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"label {i}")
        plt.imshow(val_data["label"][0, 0, :, :, 80])
        plt.subplot(1, 3, 3)
        plt.title(f"output {i}")
        plt.imshow(torch.argmax(val_outputs, dim=1).detach().cpu()[0, :, :, 80])
        plt.show()

## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)